In [1]:
import os
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
#import seaborn as sns
import networkx as nx
from functools import reduce
import plotly.graph_objects as go
import plotly.offline as py
# from addEdge import addEdge
import pulp as op
import sys
import base64
import io
import time
import plotly.express as px
from jupyter_dash import JupyterDash
import dash
# import dash_core_components as dcc
# import dash_html_components as html
from dash import html, dcc
from dash.dependencies import Input, Output
from ipywidgets import widgets
from IPython.display import clear_output
# %pylab inline

In [2]:
df = pd.read_csv('CLS_data.csv')

# Crop-Livestock System Generator and Optimization

This computer software application is a tool that aims to aid farmers and gardening enthusiasts alike in generating and optimizing their chosen crop-livestock system. It has been developed to automatically generate systems of crop/s and/or livestock/s that are mutually beneficial to each other in terms of cycling necessary nutrients that are needed in maintaining healthy crops as well as increasing the pest management capabilities of the user’s chosen system.

The user has 12 choices in filtering for crop/s and/or livestock/s that are compatible in the season the user chooses as well as the type of habitat in which their crops are cultivated and livestock are integrated in. These filtration options differentiate by 6 seasons and 2 types of habitats. 

The **seasons** are spring, summer, fall, winter, dry and wet season in which users can base their chosen temperature as such:
* Spring: 12 to 22 degree celsius when temperatures are expected to spike into warmer temperatures. (e.g. Spring season going to early summer season)
* Summer: 23 degree celsius and above.
* Fall: 12 to 22 degree celsius when temperatures are expected to fall into colder temperatures. (e.g. Fall season going to early winter season)
* Winter: 11 degrees and below.
* Dry: 22 degree celsius below or covers the season of winter and spring.
* Wet: 12 degree celsius above or covers the season of summer and fall. 

The choices in **habitation** are:
* Wetland: area of land where excess amount of water sits on top of soil in a still environment similar to shallow ponds. Examples of a crop cultivated in wetlands are rice crops.
* Well-drained soil: area of land that allows good distribution of water quickly to prevent water from pooling in one area. Example of a crop cultivated in well-drained soils is lettuce. 


In [3]:
# def dropdown_eventhandler(choice):
# #     clear_output(wait=True)
#     filter_update(radio_buttons.value)

In [4]:
def filter_update(choice):
    global comp_crop_summer_dry
    
    if choice == 'Spring and Wetland':
        comp_crop_summer_dry = df[(df.season.str.contains('Spring', case=False)) 
                                  & (df.habitat.str.contains('Wetland', case=False)) 
                                  | (df.habitat.str.contains('all habitats', case=False))]
    elif choice == 'Spring and Well-drained land':
        comp_crop_summer_dry = df[(df.season.str.contains('Spring', case=False)) 
                                  & (df.habitat.str.contains('well-drained soil', case=False)) 
                                  | (df.habitat.str.contains('all habitats', case=False))]
    elif choice == 'Summer and Wetland':
        comp_crop_summer_dry = df[(df.season.str.contains('Summer', case=False)) 
                                  & (df.habitat.str.contains('Wetland', case=False)) 
                                  | (df.habitat.str.contains('all habitats', case=False))]
    elif choice == 'Summer and Well-drained land':
        comp_crop_summer_dry = df[(df.season.str.contains('Summer', case=False)) 
                                  & (df.habitat.str.contains('well-drained soil', case=False)) 
                                  | (df.habitat.str.contains('all habitats', case=False))]
    elif choice == 'Fall and Wetland':
        comp_crop_summer_dry = df[(df.season.str.contains('Fall', case=False)) 
                                  & (df.habitat.str.contains('Wetland', case=False)) 
                                  | (df.habitat.str.contains('all habitats', case=False))]
    elif choice == 'Fall and Well-drained land':
        comp_crop_summer_dry = df[(df.season.str.contains('Fall', case=False)) 
                                  & (df.habitat.str.contains('well-drained soil', case=False)) 
                                  | (df.habitat.str.contains('all habitats', case=False))]
    elif choice == 'Winter and Wetland':
        comp_crop_summer_dry = df[(df.season.str.contains('Winter', case=False)) 
                                  & (df.habitat.str.contains('Wetland', case=False)) 
                                  | (df.habitat.str.contains('all habitats', case=False))]
    elif choice == 'Winter and Well-drained land':
        comp_crop_summer_dry = df[(df.season.str.contains('Winter', case=False)) 
                                  & (df.habitat.str.contains('well-drained soil', case=False)) 
                                  | (df.habitat.str.contains('all habitats', case=False))]
    elif choice == 'Dry season and Wetland':
        comp_crop_summer_dry = df[(df.season.str.contains('Winter', case=False))
                                 | (df.season.str.contains('Spring', case=False))
                                 & (df.habitat.str.contains('Wetland', case=False))
                                 | (df.habitat.str.contains('all habitats', case=False))]
    elif choice == 'Dry season and Well-drained land':
        comp_crop_summer_dry = df[((df.season.str.contains('Winter', case=False))
                                 | (df.season.str.contains('Spring', case=False)))
                                 & (df.habitat.str.contains('well-drained soil', case=False))
                                 | (df.habitat.str.contains('all habitats', case=False))]
    elif choice == 'Wet season and Wetland':
        comp_crop_summer_dry = df[((df.season.str.contains('Summer', case=False))
                                 | (df.season.str.contains('Fall', case=False)))
                                 & (df.habitat.str.contains('Wetland', case=False))
                                 | (df.habitat.str.contains('all habitats', case=False))]
    elif choice == 'Wet season and Well-drained land':
        comp_crop_summer_dry = df[((df.season.str.contains('Summer', case=False))
                                 | (df.season.str.contains('Fall', case=False)))
                                 & (df.habitat.str.contains('well-drained soil', case=False))
                                 | (df.habitat.str.contains('all habitats', case=False))]
    
#     comp_crop_summer_dry = df[(df.season.str.contains('Fall', case=False)) & (df.habitat.str.contains('well-drained soil', case=False)) | (df.habitat.str.contains('all habitats', case=False))]
    comp_crop_summer_dry = comp_crop_summer_dry.reset_index(drop=True)
    comp_crop_summer_dry['org_type']=comp_crop_summer_dry['org_type'].str.lower()
    comp_crop_summer_dry['root_depth']=comp_crop_summer_dry['root_depth'].str.replace(' inches', '')
    comp_crop_summer_dry['root_depth'] = comp_crop_summer_dry['root_depth'].fillna(0)
    comp_crop_summer_dry = comp_crop_summer_dry.astype({'root_depth':'int'})
    comp_crop_summer_dry['habitat']=comp_crop_summer_dry['habitat'].str.lower()
    comp_crop_summer_dry['pestAtt'] = comp_crop_summer_dry['pestAtt'].fillna("none")
    comp_crop_summer_dry['pestRep'] = comp_crop_summer_dry['pestRep'].fillna("none")
    comp_crop_summer_dry['food_source'] = comp_crop_summer_dry['food_source'].fillna("none")
    comp_crop_summer_dry['pestAtt'] = comp_crop_summer_dry['pestAtt'].str.lower()
    comp_crop_summer_dry['pestRep'] = comp_crop_summer_dry['pestRep'].str.lower()
    comp_crop_summer_dry['food_source'] = comp_crop_summer_dry['food_source'].str.lower()
    comp_crop_summer_dry['main_crop'] = comp_crop_summer_dry['main_crop'].fillna("no")
    comp_crop_summer_dry['family'] = comp_crop_summer_dry['family'].fillna("none")
    comp_crop_summer_dry['family'] = comp_crop_summer_dry['family'].str.lower()
    comp_crop_summer_dry['enemy_plant'] = comp_crop_summer_dry['enemy_plant'].fillna("none")
    comp_crop_summer_dry['enemy_plant'] = comp_crop_summer_dry['enemy_plant'].str.lower()
    root_class = []
    for row in comp_crop_summer_dry['root_depth']:
        if row > 17 and row < 25: root_class.append('moderately-deep')
        elif row < 18: root_class.append('shallow')
        elif row > 24: root_class.append('deep')
        else: root_class.append('none')

    comp_crop_summer_dry['root_class'] = root_class
    comp_crop_summer_dry['pestAtt_amnt'] = comp_crop_summer_dry['pestAtt_amnt'].fillna(0)
    comp_crop_summer_dry['pestRep_amnt'] = comp_crop_summer_dry['pestRep_amnt'].fillna(0)
    comp_crop_summer_dry['pestRep_amnt'] = np.where(comp_crop_summer_dry['pestRep_amnt'] =='n/a ', 0, comp_crop_summer_dry['pestRep_amnt'])
    comp_crop_summer_dry['pestRep_amnt'] = comp_crop_summer_dry['pestRep_amnt'].astype(float)
    def updateMult(*args):
        ms_df=comp_crop_summer_dry['org_name']
        msList=list(ms_df)
        multSelect.options=msList
    radio_buttons.observe(updateMult)

In [1]:
def listFilter(selected):
    prox_res=comp_crop_summer_dry.copy()
    global result
    result=prox_res[prox_res['org_name'].isin(selected)].reset_index(drop=True)
#     get_graph.clear_output()
    

In [ ]:
def get_graph(b):
#     listFilter(selected)
    try:
        with output:
            clear_output(wait=True)
            compList = []
            for i in result.index:
                for j in result.index: 
                    if i == j:
                        continue
            #         elif (result['main_crop'][i]=='no') & (result['main_crop'][j]=='no'):
            #             continue
                    else:    
                        if (result['org_type'][i]=='animal') & (result['org_type'][j]=='animal'):
                            string = result['org_name'][i] + ", " + result['org_name'][j]
                            if string not in compList:
                                compList.append(string)
                            else:
                                continue
                        elif (result['org_type'][i]=='animal') & (result['org_type'][j]=='plant'):
                            string = result['org_name'][i] + ", " + result['org_name'][j]
                            if string not in compList:
                                compList.append(string)
                            else:
                                continue
                        elif (result['org_type'][i]=='plant') & (result['org_type'][j]=='animal'):
                            string = result['org_name'][i] + ", " + result['org_name'][j]
                            if string not in compList:
                                compList.append(string)
                            else:
                                continue
                        elif (result['org_type'][i]=='plant') & (result['org_type'][j]=='plant'):
                            if ((result['root_depth'][i]>17) & (result['root_depth'][i]<25)) | ((result['root_depth'][j]>17) & (result['root_depth'][j]<25)):
                                string = result['org_name'][i] + ", " + result['org_name'][j]
                                if string not in compList:
                                    compList.append(string)
                                else:
                                    continue
                            elif (result['root_depth'][i]<18) & (result['root_depth'][j]>24):
                                string = result['org_name'][i] + ", " + result['org_name'][j]
                                if string not in compList:
                                    compList.append(string)
                                else:
                                    continue
                            elif (result['root_depth'][i]>24) & (result['root_depth'][j]<18):
                                string = result['org_name'][i] + ", " + result['org_name'][j]
                                if string not in compList:
                                    compList.append(string)
                                else:
                                    continue
                            else:
                                continue
            global masterList
            masterList = []
            compostList = []
            pestList = []
            for comps in compList:
                compSplit = comps.split(", ")
                #compSplit_df = result.loc[result['org_name'].isin(compSplit)].reset_index(drop=True)
                compA = result.loc[result['org_name']== compSplit[0]].reset_index(drop=True)
                compB = result.loc[result['org_name']== compSplit[1]].reset_index(drop=True)
                famA = compA['family'][0]
                famB = compB['family'][0]
                enemyA = compA['enemy_plant'][0].split(", ")
                enemyB = compB['enemy_plant'][0].split(", ")
                #condition for main crop
                if (compA['main_crop'][0]=='yes' and compB['main_crop'][0]=='yes'):
                    continue
                elif (compA['main_crop'][0]=='no' and compB['main_crop'][0]=='no'):
                    continue
                elif (famA in enemyB):
                    continue
                elif (famB in enemyA):
                    continue
                else: 
                    if (compA['org_type'][0]=='plant' and compB['org_type'][0]=='plant'): #for compost and pest
                        #for compost
            #             if (compA['end_of_harvest'][0]=='yes'):
            #                 continue
            #             else:
            #                 compTuple = tuple((compA['org_name'][0], compB['org_name'][0]))
            #                 if compTuple not in compostList:
            #                     compostList.append(compTuple)
            #                 if (compTuple not in masterList):
            #                     masterList.append(compTuple)
                #             else: 
                #                 continue
                        #for pest  
                        listA = compA['pestRep'][0].split(", ")
                        listB = compB['pestAtt'][0].split(", ")
                        for repComp in listA:
                            if repComp in listB:
                                compTuple = tuple((compA['org_name'][0], compB['org_name'][0]))
                                if (compTuple not in pestList):
                                    pestList.append(compTuple)
                                if (compTuple not in masterList):

                                    masterList.append(compTuple)
                                else: 
                                    continue
                            else:
                                 continue
                    elif (compA['org_type'][0]=='animal' and compB['org_type'][0]=='plant'): # for compost only
                        compTuple = tuple((compA['org_name'][0], compB['org_name'][0]))
                        foodList = compA['food_source'][0].split(", ")
                        if compTuple not in compostList:
                            compostList.append(compTuple)
                        if (compTuple not in masterList):
                            masterList.append(compTuple)
                #         if 'all insects' in foodList:
                #             compTuple = tuple((compA['org_name'][0], compB['org_name'][0]))
                #             if (compTuple not in pestList):
                #                 pestList.append(compTuple)
                #             if (compTuple not in masterList):
                #                 initialVertex.append(compSplit[0])
                #                 terminalVertex.append(compSplit[1])
                #                 masterList.append(compTuple)
                #             else: 
                #                 continue
                        else: 
                            continue
                    elif (compA['org_type'][0]=='plant' and compB['org_type'][0]=='animal'): # for pest only # add for food source
                        foodList = compB['food_source'][0].split(", ")
                        #allPest = 'all insects'
                        #checkList = ['all insects', 'vegetables']
                        #foodList = ['all insects']
                        if 'all insects' in foodList:
                            compTuple = tuple((compA['org_name'][0], compB['org_name'][0]))
                            if (compTuple not in pestList):
                                pestList.append(compTuple)
                            if (compTuple not in masterList):
                                masterList.append(compTuple)
                            else: 
                                continue
                        else:
                            continue
                    else:
                        continue
        #     clear_output(wait=True)
            I = nx.MultiDiGraph()
            I.add_edges_from(masterList)
            fig=plt.figure(figsize = (15,15))
            pos=nx.layout.shell_layout(I)
            I=nx.draw_networkx(I, pos=pos, with_labels=True, node_size=4000)
            """ Kosaraju's Algorithm for finding strongly connected components """

            # ==============================================================================
            # Vertex And Digraph Classes
            class Vertex:
                """ A vertex of the graph. """

                def __init__(self, value):
                    self.value = value
                    self.d = None
                    self.f = None


            class DiGraph:
                """ A simple Directed graph """

                def __init__(self): 
                    self.adj = {}

                def has_vertex(self, v):
                    """ Returns True if graph has vertex v """
                    try:
                        self.adj[v]
                        return True
                    except KeyError:
                        return False

                def add_vertex(self, v):
                    """ Adds vertex v into the graph if not already present """
                    if self.has_vertex(v):
                        return False
                    else:
                        self.adj[v] = {}
                        return True

                def add_vertices(self, arr):
                    """ Adds the vertices in arr into graph """
                    for v in arr:
                        self.add_vertex(v)
                    return True

                def has_edge(self, start, end): 
                    """ Returns True if there is an edge between start and end """
                    if self.has_vertex(start) and self.has_vertex(end):
                        try:
                            if self.adj[start][end] is True:
                                return True
                            return False
                        except KeyError:
                            return False
                    return False

                def add_edge(self, start, end):
                    """ Adds a directed edge from start to end """
                    if self.has_vertex(start) and self.has_vertex(end):
                        self.adj[start][end] = True
                        return True
                    return False

                def remove_edge(self, start, end):  
                    """ Remove an edge from the graph """
                    if self.has_edge(start, end):
                        del self.adj[start][end]
                        return True
                    return False

                def remove_vertex(self, v):
                    """ Removes vertex v from the graph """
                    if self.has_vertex(v):
                        for vertex in self.adj.keys():
                            self.remove_edge(vertex, v)
                        del self.adj[v]
                        return True
                    return False
            # ==============================================================================

            # ==============================================================================
            # Depth First Search

            def dfs_visit(G, s, parent, stack):
                """ Recursively explore all childrens of s """
                global tempTime
                tempTime = 0
                tempTime += 1
                s.d = tempTime

                for v in G.adj[s]:
                    if v not in parent:
                        parent[v] = s
                        dfs_visit(G, v, parent, stack)

                tempTime += 1
                s.f = tempTime
                stack.append(s)


            def dfs(G, stack):
                """ Explore the whole graph using Depth First Search """
                parent = {}
                stack = []

                for vertex in list(G.adj.keys()):
                    if vertex not in parent:
                        parent[vertex] = None
                        dfs_visit(G, vertex, parent, stack)

                return stack
            # ==============================================================================

            # ==============================================================================
            # Helper DFS function and Kosaraju's Algorithm

            def dfs_single_visit(adj_list, v, visited, stack):
                """ Recursively visits all the children of v """
                for u in adj_list[v]:
                    if u not in visited:
                        visited[u] = v
                        dfs_single_visit(adj_list, u, visited, stack)
                stack.append(v)

            def kosaraju(G):    
                """ Kosaraju's Algorithm for finding strongly connected components. """
                # get vertices based on their finishing tempTime in decreasing order
                stack = dfs(G, [])

                # Reverse edges of the graph G
                rev_adj = {}

                for vertex in G.adj.keys():
                    rev_adj[vertex] = {}

                for vertex in G.adj.keys():
                    for u in G.adj[vertex]:
                        rev_adj[u][vertex] = True


                # Traverse graph by popping vertices out from the stack
                visited = {}
                components = []
                i = 0

                while stack != []:
                    v = stack.pop()
                    # if v is already visited skip iteration
                    if v in visited:
                        continue
                    # otherwise find all the vertices it can reach and put them into components
                    else:
                        components.append([])
                        if v not in visited:
                            visited[v] = True
                            dfs_single_visit(rev_adj, v, visited, components[i])

                        components.append([])
                        i += 1

                return components
            # ==============================================================================

            # ==============================================================================
            # Function to print user friendly data

            def print_stack(stk):
                for i in stk:
                    print(i.value, i.f)

            def print_adjacency_list(adj):
                for v in adj.keys():
                    print(v.value, " -> ", end=" ")
                    for u in adj[v]:
                        print(u.value, end=" ")
                    print()
            # ==============================================================================


            G = DiGraph()

            #for getting list of vertices <-- working
            len_df = len(result.index)
            vList = []
            for i in range(len_df):
                vList.append(Vertex(result['org_name'][i])) #check if right dtype
                #vList.append(result['org_name'][i])
                #vList.append(Vertex(result['org_name'][i]), name=result['org_name'][i])

            G.add_vertices(vList)

            len_mList = len(masterList)
            for i in range(len_mList):
                frListA = masterList[i][0]
                frListB = masterList[i][1]
                compA = result.index[result['org_name']==frListA].tolist()[0]
                compB = result.index[result['org_name']==frListB].tolist()[0]
                G.add_edge(vList[compA], vList[compB])
            global y
            global compDrop
            global ogDrop
            compDrop=[]
            ogDrop=[]
            y = kosaraju(G)
        #     compList=[]
            for j in range(len(y)):
                if y[j] != []:
                    compString='Component '+str(j+1)+': '
                    if len(y[j])>1:
                        for v in y[j]:
                            compString = compString+" "+str(v.value)
                        compDrop.append(compString)
                        ogDrop.append(compString)
                    else:
                        for v in y[j]:
                            compString = compString+" "+str(v.value)
                        ogDrop.append(compString)
            choiceList = []
            for v in y[ogDrop.index(compDrop[0])]:
                choiceList.append(v.value)
    #     def update(*args):
    #         compCons.options=choiceList
    #     dropDown.observe(update)
        def update(*args):
            dropDown.options=compDrop
            compCons.options=choiceList
    #     multSelect.observe(update)
        button.on_click(update)
    except:
        print('No connections generated. Change selected crop/s and/or livestock/s')

In [1]:
def moo_result(choiceComp):
    try:
        global choiceList
        global x
        global choiceLen
        choiceList = []
        for v in y[ogDrop.index(choiceComp)]:
            choiceList.append(v.value)
    #     def update(*args):
    #         compCons.options=choiceList
    #     dropDown.observe(update)
    #     print(mmSlider)
        x = {}
        choiceLen = len(choiceList)
        for i in range(choiceLen):
            x[i] = op.LpVariable(choiceList[i] ,lowBound = 1)
        print('Components of your chosen crop-livestock system:')
        for i in range(choiceLen):
            print(choiceList[i])
    except:
        print('No connections generated. Change selected crop/s and/or livestock/s')

In [1]:
def moo_result2(landCon, indivCons, lbVal, ubVal, interval):
    try:
        index = choiceList.index(indivCons)
        x[index].lowBound=lbVal
        x[index].upBound=ubVal
        for i in range(len(x)):
            print(str(x[i]))
            print('lower bound: '+str(x[i].lowBound)+ ' and upper bound: '+str(x[i].upBound))
    #         print('upper bound: '+str(x[i].upBound))
            # a function for calculating the value of each objective given the value of decision variables

        def obj_val(k, a):
    #             comp = result.loc[result['org_name']== choiceList[0]].reset_index(drop=True)
    #             eqtn1 = (comp['compMat_amnt'][0])*a[0]
    #             if comp['org_type'][0]=='animal':
    #                 eqtn2 = (comp['consump_amount'][0]*100000)*a[0]
    #             else:
    #                 eqtn2 = ((comp['pestRep_amnt'][0]))*a[0]
    #             for i in range(choiceLen-1):
    #                 forComp = result.loc[result['org_name']== choiceList[i+1]].reset_index(drop=True)
    #                 eqtn1 = eqtn1+(forComp['compMat_amnt'][0])*a[i+1]
    #     #             eqtn2 = eqtn2 + (forComp['pestAtt_amnt'][0]-comp['pestRep_amnt'][0])*a[i+1]
    #                 if comp['org_type'][0]=='animal':
    #                     eqtn2 = eqtn2 + (forComp['consump_amount'][0]*100000)*a[i+1]
    #                 else:
    # #                     eqtn2 = eqtn2 + ((forComp['pestRep_amnt'][0]*10)-comp['pestAtt_amnt'][0])*a[i+1]
    #                     eqtn2 = eqtn2 + ((forComp['pestRep_amnt'][0]))*a[i+1]
    #             if k ==0:
    #                 return eqtn1
    #             if k ==1:
    #                 return eqtn2
            if k ==0:
                comp = result.loc[result['org_name']== choiceList[0]].reset_index(drop=True)
                eqtn1 = (comp['compMat_amnt'][0])*a[0]
                for i in range(choiceLen-1):
                    forComp = result.loc[result['org_name']== choiceList[i+1]].reset_index(drop=True)
                    eqtn1 = eqtn1+(forComp['compMat_amnt'][0])*a[i+1]
                return eqtn1
            if k ==1:

        #         if comp['org_type'][0].lower()=='animal':
        #             eqtn2 = (comp['consump_amount'][0])*a[0]
        #         elif comp['org_type'][0].lower()=='plant':
        #             eqtn2 = (comp['pestRep_amnt'][0])*a[0]
                for i in range(choiceLen):
                    comp = result.loc[result['org_name']== choiceList[i]].reset_index(drop=True)
                    if i==0:
                        if comp['org_type'][0].lower()=='animal':
                            eqtn2 = (comp['consump_amount'][0]*100)*a[0]
                        elif comp['org_type'][0].lower()=='plant':
                            eqtn2 = (comp['pestRep_amnt'][0]-comp['pestAtt_amnt'][0])*a[0]
                    else:
        #                 forComp = result.loc[result['org_name']== choiceList[i+1]].reset_index(drop=True)
        #                 eqtn2 = eqtn2 +(forComp['pestRep_amnt'][0])*a[i]
                        if comp['org_type'][0].lower()=='animal':
                            eqtn2 = eqtn2 + (comp['consump_amount'][0]*100)*a[i]
                        elif comp['org_type'][0].lower()=='plant':
                            eqtn2 = eqtn2 + (comp['pestRep_amnt'][0]-comp['pestAtt_amnt'][0])*a[i]
                return eqtn2

        # objective Functions
        obj = {
               0: obj_val(0, x),
               1: obj_val(1, x),
               }

        # constraints
        # cons = {
        #         0: 5*x[0]+4*x[1]+x[2] <= 100000,
        #         1: 3*5*x[0]+4*4*x[1]+5*2*x[2] <=100000,
        #         2: 3*5*x[0]+4*4*x[1]+5*2*x[2] >=0,
        #         3: 3*2*x[0]+4*1*x[1]+5*1*x[2] >=0,
        #         4: 3*2*x[0]+4*1*x[1]+5*1*x[2] <=100000
        #        }
        cons = {}
        #for land constraint
        comp = result.loc[result['org_name']== choiceList[0]].reset_index(drop=True)
        con1 = (comp['landCons'][0])*x[0]
        #for carbon constraint
        con2 = (comp['carbon_cont'][0])*x[0]
        #for nitrogen constraint
        con3 = (comp['nitro_cont'][0])*x[0]
        #for pest management constraint
        #     con4 = 
        #     con5
        for i in range(choiceLen-1):
            forComp = result.loc[result['org_name']== choiceList[i+1]].reset_index(drop=True)
            con1 = con1+(forComp['landCons'][0])*x[i+1]
            con2 = con2+(forComp['carbon_cont'][0])*x[i+1]
            con3 = con3+(forComp['nitro_cont'][0])*x[i+1]

        cons[0]=con1<=landCon
        #carbon
        cons[1]=con2>=0
        cons[2]=con2<=landCon
        #nitrogen
        cons[3]=con3>=0
        cons[4]=con3<=landCon
        ##### Generating payoff table using lexicographic optimization
        payoff=np.zeros([len(obj),len(obj)]);
        for k in range(0,len(obj)):
            model = op.LpProblem("Max",op.LpMaximize)
            model += obj[k]
            for i in range(0,len(cons)):
                model += cons[i]
            resultmoo = model.solve()
            if op.LpStatus[resultmoo] == 'Optimal':
    #             print(str(op.LpStatus[result])+" ; max value = "+str(op.value(model.objective)))
    #             for i in range(len(x)):
    #                 print(choiceList[i]+" = "+str(op.value(x[i])))
                payoff[k,k]= op.value(model.objective);
                kp=k+1;
                model = op.LpProblem("Max",op.LpMaximize)
                while kp<= len(obj)-1:
                        model += obj[kp]
                        if kp-1>=0:
                            model += obj[kp-1] >= payoff[k,kp-1]
                        for i in range(0,len(cons)):
                            model += cons[i]
                        resultmoo = model.solve()
                        if op.LpStatus[resultmoo] == 'Optimal':
    #                         print(str(op.LpStatus[result])+" ; max value = "+str(op.value(model.objective)))
    #                         for i in range(len(x)):
    #                             print(choiceList[i]+" = "+str(op.value(x[i])))
                            payoff[k,kp]= op.value(model.objective)
                            kp += 1     
                        else:
    #                             sys.exit('no optimal solution for mod_payoff')
    #                         print('no optimal solution for mod_payoff')
                            break
                kp=0;
                model += obj[k] >= payoff[k,k]
                while kp< k:
                    model += obj[kp]
                    if kp-1>=0:
                        model += obj[kp-1] >= payoff[k,kp-1]
                    for i in range(0,len(cons)):
                        model += cons[i]
                    resultmoo = model.solve()
                    if op.LpStatus[resultmoo] == 'Optimal':
    #                     print(str(op.LpStatus[result])+" ; max value = "+str(op.value(model.objective)))
    #                     for i in range(len(x)):
    #                         print(choiceList[i]+" = "+str(op.value(x[i])))
                        payoff[k,kp]= op.value(model.objective)
                        kp += 1
                    else:
    #                         sys.exit('no optimal solution for mod_payoff')
    #                     print('no optimal solution for mod_payoff')
                        break

            else:
    #                 sys.exit('no optimal solution for mod_payoff')
    #             print('no optimal solution for mod_payoff')
                break
        minobj=np.zeros([len(obj),1]);
        maxobj=np.zeros([len(obj),1]);
        for k in range(0,len(obj)):
                minobj[k] = min(payoff[:,k]);
                maxobj[k] = max(payoff[:,k]);
        # slack variables
        s = {
             1: op.LpVariable("s2",lowBound = 0),
             }
        intervals=interval;    
        solutions=[]
        lst = np.empty([intervals+1,len(obj)]);
        for g in range(0,intervals+1):
            gridpoint = []
            gpString = 'grid point no: '+str(g+1)+ ' value: '+str(maxobj[1] - ((g)/intervals)*(maxobj[1]- minobj[1]))
        #     print('grid point no: ', g+1, 'val: ', maxobj[1] - ((g)/intervals)*(maxobj[1]- minobj[1]))
            gridpoint.append(gpString)
            model = op.LpProblem("Max",op.LpMaximize)
            for i in range(len(x)):
                varString = choiceList[i]+" = "+str(int(op.value(x[i])))
                gridpoint.append(varString)
        #         print(choiceList[i]+" = "+str(op.value(x[i])))
            solutions.append(gridpoint)
            code = 1/(maxobj[1]-minobj[1]);
            model += obj[0]+1e-3*s[1]*code
            model += obj[1]- s[1] == maxobj[1] - ((g)/intervals)*(maxobj[1]- minobj[1])

            for i in range(0,len(cons)):
                model += cons[i]
            resultmoo = model.solve()
            if op.LpStatus[resultmoo] == 'Optimal':
        #         d = [op.value(x[0]), op.value(x[1]), op.value(x[2])]; #change this part
                d = []
                for i in range(len(x)):
                    d.append(op.value(x[i]))
                for k in range(0,len(obj)):
                    lst[g,k]=obj_val(k, d);
            else:
    #             print('early exit (jump)')
                break
        def update_gpList(*args):
            gpDrop.options=solutions
        intGP.observe(update_gpList)
        compCons.observe(update_gpList)
    #     consSlider.observe(update_gpList)
        lBound.observe(update_gpList)
        uBound.observe(update_gpList)
        land.observe(update_gpList)
    #     ax=plt.scatter(lst[:,0], lst[:,1])
        # set axis lables
    #     plt.xlabel("Z1")
    #     plt.ylabel("Z2")
    #     # set chart title
    #     plt.title("Pareto-Optimal Solutions")
    #     plt.show()
        # lst[:,0]
        # lst[:,1]
    #     print("Z2: "+ obj[0])
    #     print("Z1:"+ obj[1])
        fig= px.scatter(x=lst[:,0], 
                        y=lst[:,1],
                        labels=dict(x="Amount of Compost Material (in kg)", 
                                    y="Pest Management")
                       )
        fig.show()
    
    except:
        print('cannot be optimized, try adjusting constraint selection.')

In [7]:
def display_GP(gpChoice):
    try:
#     print(gpChoice)
        for g in gpChoice:
            print(g)
    except:
        print("No gridpoints selected. Please select gridpoint in the dropdown selection or adjust constraints above.")

In [8]:
filterOp = ['Spring and Wetland', 'Spring and Well-drained land', 'Summer and Wetland', 'Summer and Well-drained land',
           'Fall and Wetland', 'Fall and Well-drained land', 'Winter and Wetland', 'Winter and Well-drained land']
radio_buttons = widgets.Dropdown(
    options=filterOp,
    value='Spring and Wetland',
    description='choice: ',
)


In [1]:
widgets.interact(filter_update, choice=radio_buttons)
dropDown = widgets.Dropdown(
#     options = compDrop,
    description="dropdown: ",
#     continuous_update=True
)
multSelect = widgets.SelectMultiple(
#     options=['Apples', 'Oranges', 'Pears'],
#     value=['Oranges'],
    #rows=10,
    description='components',
    disabled=False
)
intGP=widgets.IntSlider(
    value=7,
    min=0,
    max=100,
    step=1,
    description='Intervals:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)
compCons = widgets.Dropdown(
#     options = compDrop,
    description="component: ",
#     continuous_update=True
)
# consSlider=widgets.IntRangeSlider(
#     value=[5, 7],
#     min=0,
#     max=10,
#     step=1,
#     description='component constraint:',
# #     disabled=False,
# #     continuous_update=False,
#     orientation='horizontal',
#     readout=True,
#     readout_format='d',
# )
lBound=widgets.IntText(
    value=1,
    description='lower bound:',
    disabled=False
)
uBound=widgets.IntText(
    value=10000,
    description='upper bound:',
    disabled=False
)
land=widgets.IntText(
    value=400,
    description='land area(sqm):',
    disabled=False
)
gpDrop = widgets.Dropdown(
#     options = compDrop,
    description="gridpoints: ",
#     continuous_update=True
)

NameError: name 'widgets' is not defined

In [ ]:
widgets.interact(listFilter, 
                 selected=multSelect)

In [ ]:
button = widgets.Button(description="Generate!")
output = widgets.Output()

display(button, output)

# def on_button_clicked(b):
#     with output:
#         print("Button clicked.")

button.on_click(get_graph)

### Optimization 

<p>The resulting crop-livestock system is listed in the drowdown list below. Listed systems will only include systems that contain more than 1 component to ensure that there is a mutual benefical relationship in the system. Please select a system to be optimized. </p>

In [10]:
widgets.interact(moo_result, 
                 choiceComp=dropDown,  
#                  interval=(1, 10, 1),
#                  indivCons=compCons,
#                  mmSlider=consSlider
                )

interactive(children=(Dropdown(description='dropdown: ', options=(), value=None), Output()), _dom_classes=('wi…

<function __main__.moo_result(choiceComp)>

<p>You are free set constraints and conditions for the resulting optimization. The following selections for setting constraints are:</p>
<br>
<li><strong>Land Text Entry:</strong> in this section, you may input the available land that is available for setting your crop-livestock system. The unit of measurement for this section is in area per sqaure meter (sqm). Please indicate the available land area following the unit of measurement mentioned. </li>
<br>
<li><strong>Component dropdown and boundary input:</strong> in the dropdown list for component, you may choose a component in the crop-livestock system to set corresponding boundary in value. The minimun amount of the selected component will be indicated on the lower bound text entry box and the maximum amount of the selected component will be indicated on the upper bound text entry box. Adjustments for the value boundary for each component will automatically reflect in the plot below.</li>
<br>
<li><strong>Interval Value Slider:</strong> for this slider, you may adjust the number of optimal solutions that the application may generate for you. The given plot below will adjust automatically in relation to your chosen value. </li>
<br>

In [11]:
widgets.interact(moo_result2, 
#                  choiceComp=dropDown,
                 landCon=land,
#                  interval=intGP,
                 indivCons=compCons,
                 lbVal=lBound,
                 ubVal=uBound,
                 interval=intGP
                )

interactive(children=(IntText(value=200, description='land available:'), Dropdown(description='component: ', o…

<function __main__.moo_result2(landCon, indivCons, interval, mmSlider)>

In [12]:
widgets.interact(display_GP,
                gpChoice=gpDrop
                )

interactive(children=(Dropdown(description='gridpoints: ', options=(), value=None), Output()), _dom_classes=('…

<function __main__.display_GP(gpChoice)>

In [13]:
# radio_buttons.observe(dropdown_eventhandler, names='value')
# display(radio_buttons, dropDown)

In [14]:
# ogDrop